In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm
import glob
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

def load_image(path, IMAGE_SIZE=128, is_mask=False):
    # Load Image
    image = load_img(path, color_mode="grayscale" if is_mask else "rgb")
    # Convert to Array
    image = img_to_array(image)
    # Resize
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    # Standardization
    image = image / 255.0
    if is_mask:
        # Ensure the mask has a single channel
        image = image[..., 0]
        image = tf.expand_dims(image, axis=-1)
    return image

def load_images(paths, IMAGE_SIZE=128, is_mask=False):
    # Create Space
    channels = 1 if is_mask else 3
    images = np.zeros((len(paths), IMAGE_SIZE, IMAGE_SIZE, channels), dtype=np.float32)
    # Iterate Paths
    for i, path in tqdm(enumerate(paths), desc="Loading", total=len(paths)):
        # Load Image
        images[i] = load_image(path, IMAGE_SIZE=IMAGE_SIZE, is_mask=is_mask)
    return images

def load_data(root_path, trim=None, IMAGE_SIZE=128, BATCH_SIZE=8, files=False):
    # Collect Image Paths
    image_paths = sorted(glob.glob(os.path.join(root_path, "images", "*.png")))
    # Trim if needed
    if trim is not None:
        image_paths = image_paths[trim[0]:trim[1]]
    # Update mask paths
    mask_paths = [path.replace("images", "labels") for path in image_paths]
    # Load Data
    images = load_images(image_paths, IMAGE_SIZE=IMAGE_SIZE, is_mask=False)
    masks = load_images(mask_paths, IMAGE_SIZE=IMAGE_SIZE, is_mask=True)
    # Convert To Dataset
    data = tf.data.Dataset.from_tensor_slices((images, masks))
    data = data.batch(BATCH_SIZE, drop_remainder=True)
    data = data.shuffle(1000).prefetch(tf.data.AUTOTUNE)
    if files:
        return data, images, masks
    return data

train_ds = load_data(root_path="/kaggle/input/dataset-itupak/AI4GLOF_Dataset/train", trim=[0, 1000], IMAGE_SIZE=128, BATCH_SIZE=8)
valid_ds, val_images, val_masks = load_data(root_path="/kaggle/input/dataset-itupak/AI4GLOF_Dataset/test", trim=[0, 200], files=True, IMAGE_SIZE=128, BATCH_SIZE=8)

In [ ]:
def show_images(data, model=None, explain=False, n_images=6, SIZE=(25,10)):
    # Plot Configurations
    if model is not None:
        if explain:
            n_cols = 5
        else:
            n_cols = 4
    else:
        n_cols = 3

    # Iterate through data
    for plot_no in range(1, n_images+1):
        for index, (images, masks) in enumerate(data):

            # Select Items
            id = np.random.randint(len(images))
            image, mask = images[id], masks[id]

            if model is not None:

                if explain:
                    # Make Prediction
                    pred_mask = model.predict(image[np.newaxis, ...])[0]

                    # Figure
                    plt.figure(figsize=SIZE)

                    # Original Image
                    plt.subplot(1, n_cols, 1)
                    plt.imshow(image)
                    plt.axis('off')
                    plt.title("Original Image")

                    # Original Mask
                    plt.subplot(1, n_cols, 2)
                    plt.imshow(mask[..., 0], cmap='gray')
                    plt.axis('off')
                    plt.title('Original Mask')

                    # Predicted Mask
                    plt.subplot(1, n_cols, 3)
                    plt.imshow(pred_mask[..., 0], cmap='gray')
                    plt.axis('off')
                    plt.title("Predicted Mask")

                    # Mixed
                    plt.subplot(1, n_cols, 4)
                    plt.imshow(image)
                    plt.imshow(pred_mask[..., 0], alpha=0.4, cmap='gray')
                    plt.axis('off')
                    plt.title("Overlapped Prediction")

                    # Show Image
                    plt.show()

                else:
                    # Make Prediction
                    pred_mask = model.predict(image[np.newaxis, ...])[0]

                    # Figure
                    plt.figure(figsize=SIZE)

                    # Original Image
                    plt.subplot(1, n_cols, 1)
                    plt.imshow(image)
                    plt.axis('off')
                    plt.title("Original Image")

                    # Original Mask
                    plt.subplot(1, n_cols, 2)
                    plt.imshow(mask[..., 0], cmap='gray')
                    plt.axis('off')
                    plt.title('Original Mask')

                    # Predicted Mask
                    plt.subplot(1, n_cols, 3)
                    plt.imshow(pred_mask[..., 0], cmap='gray')
                    plt.axis('off')
                    plt.title("Predicted Mask")

                    # Mixed
                    plt.subplot(1, n_cols, 4)
                    plt.imshow(image)
                    plt.imshow(pred_mask[..., 0], alpha=0.4, cmap='gray')
                    plt.axis('off')
                    plt.title("Overlapped Prediction")

                    # Show Image
                    plt.show()

            else:
                # Figure
                plt.figure(figsize=SIZE)

                # Original Image
                plt.subplot(1, n_cols, 1)
                plt.imshow(image)
                plt.axis('off')
                plt.title("Original Image")

                # Original Mask
                plt.subplot(1, n_cols, 2)
                plt.imshow(mask[..., 0], cmap='gray')
                plt.axis('off')
                plt.title('Original Mask')

                # Mixed
                plt.subplot(1, n_cols, 3)
                plt.imshow(image)
                plt.imshow(mask[..., 0], alpha=0.4, cmap='gray')
                plt.axis('off')
                plt.title("Overlapped Image")

                # Show Image
                plt.show()

            # Break Loop
            break

show_images(train_ds, n_images=3, SIZE=(15,5))
show_images(valid_ds, n_images=3, SIZE=(15,8))

In [ ]:
def unet_model(input_size=(128, 128, 3)):
    inputs = Input(input_size)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = Concatenate()([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = Concatenate()([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = Concatenate()([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = Concatenate()([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [ ]:
import tensorflow.keras.backend as K
import tensorflow as tf

def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    total = K.sum(y_true_f) + K.sum(y_pred_f)
    union = total - intersection
    return (intersection + smooth) / (union + smooth)

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

In [ ]:
model = unet_model(input_size=(128, 128, 3))
model.summary()

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy', dice_coefficient, iou, precision, recall, f1_score])

In [ ]:
class ShowProgress(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1) % 5 == 0:
            show_images(valid_ds, model=self.model, explain=False, n_images=5, SIZE=(15,8))

callbacks = [ModelCheckpoint("UNet.keras"), ShowProgress()]

history = model.fit(train_ds, validation_data=valid_ds, epochs=50, callbacks=callbacks)

show_images(valid_ds, model=model, explain=False, n_images=10, SIZE=(20,8))

model.save("UNet_Sentinel2.keras")

In [ ]:
# Assuming `history` is the object returned by model.fit()
import matplotlib.pyplot as plt

def print_evaluation_metrics(history):
    # Get the metrics from the history object
    metrics = history.history

    # Print the final values of the metrics
    print("Final Training Metrics:")
    print(f"Accuracy: {metrics['accuracy'][-1]:.4f}")
    print(f"Dice Coefficient: {metrics['dice_coefficient'][-1]:.4f}")
    print(f"IoU: {metrics['iou'][-1]:.4f}")
    print(f"Precision: {metrics['precision'][-1]:.4f}")
    print(f"Recall: {metrics['recall'][-1]:.4f}")
    print(f"F1 Score: {metrics['f1_score'][-1]:.4f}")
    
    print("\nFinal Validation Metrics:")
    print(f"Val Accuracy: {metrics['val_accuracy'][-1]:.4f}")
    print(f"Val Dice Coefficient: {metrics['val_dice_coefficient'][-1]:.4f}")
    print(f"Val IoU: {metrics['val_iou'][-1]:.4f}")
    print(f"Val Precision: {metrics['val_precision'][-1]:.4f}")
    print(f"Val Recall: {metrics['val_recall'][-1]:.4f}")
    print(f"Val F1 Score: {metrics['val_f1_score'][-1]:.4f}")

# Call the function to print the metrics
print_evaluation_metrics(history)

# Optionally, you can also plot the metrics
def plot_metrics(history):
    metrics = ['accuracy', 'dice_coefficient', 'iou', 'precision', 'recall', 'f1_score']
    
    for metric in metrics:
        plt.figure()
        plt.plot(history.history[metric], label='train_' + metric)
        plt.plot(history.history['val_' + metric], label='val_' + metric)
        plt.xlabel('Epochs')
        plt.ylabel(metric)
        plt.title(f'Training and Validation {metric}')
        plt.legend()
        plt.show()

plot_metrics(history)

In [ ]:
from IPython.display import FileLink

FileLink(r"UNet_Sentinel2.keras")